In [ ]:
$find ~/Playing/waxholm/scenes_formatted/ -name '*.mix'|while read i;do cat $i |grep -v '^$'|grep -v '^FR'|grep -v '^CT'|grep -v '^WIZARD'|grep -v '^AUTOLAB'|grep -v 'CORRECTED:'|grep -v 'DATA BANK MATERIAL'|grep -v '^SPEAKER'; done

In [ ]:
%writefile kludge.pl
#!/usr/bin/perl

use warnings;
use strict;
use utf8;

binmode(STDIN, ":utf8");
binmode(STDOUT, ":utf8");
binmode(STDERR, ":utf8");

my $smp = '';
my $last = '';
my $text = '';
my $phone = '';
my $label = '';

sub clean_up {
        my $text = shift;

        $text =~ s/\s+/ /g;
        $text =~ s/\{/ä/g;
        $text =~ s/\}/å/g;
        $text =~ s/\|/ö/g;
        $text =~ s/\\/Ö/g;
        return $text;
}

while(<>) {
        chomp;
        if(m!Waxholm dialog. /u/wax/data/scenes/[^/]*/(.*)!) {
                my $newsmp = $1;
                $last = 'smp';
                $text = clean_up($text);
                $phone = clean_up($phone);
                $label = clean_up($label);

                if ($smp ne '') {
                        print "$smp\t$text\t$phone\t$label\n";
                }
                $smp = $newsmp;
                $text = '';
                $phone = '';
                $label = '';
        } elsif (m!^PHONEME:!) {
                s/^PHONEME:\s+//;
                $phone = $_;
                $last = 'phone';
        } elsif (m!^Labels:!) {
                s/^Labels:\s+//;
                $label = $_;
                $last = 'label';
        } elsif (m!^TEXT:!) {
                s/^TEXT:\s*//;
                $text = $_;
                $last = 'text';
        } elsif (m!^S{g f|ljande mening:!) {
                print STDERR "$smp:$_\n";
        } else {
                if($last eq 'text') {
                        $text = "$text $_";
                } elsif($last eq 'label') {
                        $label = "$label $_";
                } elsif($last eq 'phone') {
                        $phone = "$phone $_";
                } else {
                        print "DAFUQ: $_";
                }
        }
}